In [1]:
!pip install transformers
!pip install sentencepiece
!pip install bert
!pip install accelerate -U
!pip install evaluate
!pip install datasets

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert: filename=bert-2.2.0-py3-none-any.whl size=3744 sha256=ac719e0d84c2b5105b1c610c223ea7fd3a63d0b4319236ff205b9b0173e1e657
  Stored in directory: /root/.cache/pip/wheels/d0/97/c8/5775d57c323970511488d9b53ce66cf0ded394fe4b4f6e0afb
  Created wheel for erlastic: filename=erlastic-2.0.0-py3-none-any.whl size=6779 sha256=7c0b8fb49b101ac9fe0618d7d1fb54219b54059ba787249ad0ed2889eab7d17f
  Stored in directory: /root/.cache/pip/wheels/f5/ca/b1/ce71c2e0e9e0df3786697d06d53574be03ef86cd4fb0e2cb99
Successfully built bert erlastic
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import pandas as pd
import numpy as np
from transformers import BertModel, BertForMultipleChoice, BertTokenizer, AdamW
import tensorflow as tf
import torch

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMultipleChoice.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import torch

## FINE TUNING BERT MODEL

In [5]:
training_data = pd.read_csv('train_data.csv')

In [6]:
testing_data = pd.read_csv('test_data.csv')

## Finding the labels for the Training Data

In [7]:
def finding_labels():
  labels = []
  for index, item in training_data.iterrows():
    if item['label_A'] == "stereotype":
      labels.append(0)
    elif item['label_B'] == "stereotype":
      labels.append(1)
    else:
      labels.append(2)
  return labels

In [8]:
label = finding_labels()

In [9]:
print(len(label))

80


In [10]:
first, second = [], []
sentence_type = ['sentence_A', 'sentence_B', 'sentence_C']
for index, item in training_data.iterrows():
  question = item['context'][:-1] if item['context'][-1] == '.' else item['context']
  prev_q = "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. "
  question = prev_q + "The stereotypical reason for this statement: " + question + " is "
  first_sentence = [question for i in range(3)]
  first = first + first_sentence
  second_sentence = [f"{question} {item[i]}" for i in sentence_type]
  second.append(second_sentence)

second = sum(second, [])
tokenized_examples = tokenizer(first, second, truncation=True)
element = {k: [v[i : i + 3] for i in range(0, len(v), 3)] for k, v in tokenized_examples.items()}

In [11]:
print(first)

["Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. The stereotypical reason for this statement: Many people live in Ethiopia is ", "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. The stereotypical reason for this statement: Many people live in Ethiopia is ", "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. The stereotypical reason for this statement: Many people live in Ethiopia is ", "Keep in mind that the following words are potential descriptions of bias: ['nurturing', 'quiet', 'smart', 'kind', 'war', 'athletic', 'day', 'strong', 'people', 'good']. The stereotypical reason for this statement: My professor is a hispanic man is 

In [12]:
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """

    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])

        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [13]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.6 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [14]:
import evaluate
accuracy = evaluate.load("accuracy")

In [15]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [16]:
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer

In [32]:
from huggingface_hub import notebook_login

notebook_login()

In [33]:
training_args = TrainingArguments(
    output_dir="bert_bias_model_2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=10e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
)

In [34]:
import pandas as pd
from torch.utils.data import Dataset

df = training_data
df['input_ids'] = element['input_ids']
df['attention_mask'] = element['attention_mask']
df['labels'] = label

In [35]:
import datasets
from datasets import Dataset, DatasetDict

In [36]:
from sklearn.model_selection import train_test_split

df_train, df_valid = train_test_split(df, test_size=0.2, random_state=42)

In [37]:
td_train = Dataset.from_pandas(df_train)
td_valid = Dataset.from_pandas(df_valid)

In [38]:
ds = DatasetDict()

ds['train'] = td_train
ds['validation'] = td_valid

In [39]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.090396,0.437500
2,No log,0.955923,0.500000
3,No log,0.948491,0.625000
4,No log,0.901983,0.625000
5,No log,0.907644,0.687500


TrainOutput(global_step=30, training_loss=0.8330554962158203, metrics={'train_runtime': 2080.8405, 'train_samples_per_second': 0.154, 'train_steps_per_second': 0.014, 'total_flos': 85123884329568.0, 'train_loss': 0.8330554962158203, 'epoch': 5.0})

In [41]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice

In [42]:
tokenizer_new = AutoTokenizer.from_pretrained("bert_bias_model_2")

In [43]:
model_new = AutoModelForMultipleChoice.from_pretrained("bert_bias_model_2")

In [44]:
data_store = pd.DataFrame(columns = ['id','question', 'bias_type','score','predicted_type','predicted_answer', 'target'])

In [45]:
bias = ['stereotype', 'unrelated', 'anti-stereotype']
count = 0
for index, item in testing_data.iterrows():
  id = item['id']
  bias_type = item['bias_type']
  options = [item["sentence_A"], item["sentence_B"], item["sentence_C"]]
  type_ans = [item['label_A'], item['label_B'], item['label_C']]
  question = "Which option is a stereotype based on this context: " + item['context'].split('.')[0] + "?"


  encoded_input = tokenizer_new([[question, options[0]],
                          [question, options[1]],
                          [question, options[2]]], return_tensors="pt", padding=True)

  labels = torch.tensor(0).unsqueeze(0)
  outputs = model_new(**{k: v.unsqueeze(0) for k, v in encoded_input.items()}, labels=labels)
  logits = outputs.logits

  predicted_class = logits.argmax().item()
  predicted_answer = options[predicted_class]
  predicted_type = type_ans[predicted_class]
  score = 0

  if predicted_type == "stereotype":
    score = 1

  new_row = {'id': id, 'question' : question, 'bias_type': bias_type, 'score': score, 'predicted_type': predicted_type, 'predicted_answer': predicted_answer, 'target': item['target']}
  data_store = data_store.append(new_row, ignore_index=True)
  # print(data_store)
  # count += 1
  # if count == 4:
  #   break


<ipython-input-45-5f09950723c7>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-45-5f09950723c7>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-45-5f09950723c7>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-45-5f09950723c7>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_store = data_store.append(new_row, ignore_index=True)
<ipython-input-45-5f09950723c7>:28: FutureWarning: The frame.append method is de

In [51]:
data_store.to_csv('bert_ft_addition_of_positive_bias.csv',index=False)

In [52]:
print(data_store['score'].value_counts().get(0, 0))
print(data_store['score'].value_counts().get(1, 1))